## Latent Dirichlet Allocation
This is a demostration of LDA topic model using Gibbs sampling on a "perfect dataset"   
Thanks to the clear [tutorial](https://www.cnblogs.com/pinard/p/6831308.html) provided by Pinard Liu  
Author: kUNQI jIANG   
Date: 2019/1/22  

### Corpus generation
As Gibbs sampling in LDA essentially based on bag-of-words so the order of words does not matter, I use completely seperated wordset of different topic to generate pure topic documents as corpus. This is the extreme case where words and topics will be completely clustered after LDA as we can see in the result. While in real word, a word exist in different topics, and a document can cover multi-topics.

In [1]:
food_set = ["broccoli","banana","spinach","smoothie","breakfast","ham","cream","eat","vegetable","dinner","lunch",
            "apple","peach","pork","beef","rice","noodle","chicken","KFC","restaurant","cream","tea","pan","beacon"]
animal_set = ["dog","cat","fish","chinchilla","kitten","cute","hamster","munching","bird","elephant","monkey","zoo",
              "zoology","pig","piggy","duck","mice","micky","tiger","lion","horse","dragon","panda","bee","rabbit"]
soccer_set = ["football","pitch","play","player","cup","ballon","messi","ronald","manU","liverpool","chelase","ozil",
              "practice","hard","dream","stadium","fast","speed","strong","move","shot","attack","defense","win"]

In [2]:
import numpy as np
def generate(topic_set):
    sent = np.random.choice(topic_set,10)
    return " ".join(sent)

In [3]:
topics_set = [food_set,animal_set,soccer_set]
corpus = []
for i in range(100):
    corpus.append(generate(topics_set[0]).split())
    corpus.append(generate(topics_set[1]).split())
    corpus.append(generate(topics_set[2]).split())

In [4]:
import numpy as np

all_words = [word for document in corpus for word in document]
vocab = set(all_words)
num_docs = len(corpus)
num_words = len(vocab)
word2id = {w:i for i,w in enumerate(vocab)}
id2word = {i:w for i,w in enumerate(vocab)}

In [5]:
# model 3 latent topics 
num_topics = 3
# Dirichlet prior
alpha = np.ones([num_topics])
ita = 0.1 * np.ones([num_words])

### Random assignment
At the start randomly assign topic to each word in each document

In [6]:
topic_assignments = []
docs_topics = np.zeros([num_docs,num_topics]) # counts of topic assignments of each word in each doc
words_topics = np.zeros([num_words,num_topics]) # counts of topic distributes of each word over all doc
topics_words = np.zeros([num_topics,num_words]) # counts of word distributes of each topic over all doc

for d,document in enumerate(corpus):
    theta = np.random.dirichlet(alpha, 1)[0]
    doc_topics = []
    for n,word in enumerate(document):
        sample = np.random.multinomial(1, theta, size=1)[0]
        topic = list(sample).index(1)
        doc_topics.append(topic)
        docs_topics[d,topic] += 1
        words_topics[word2id[word],topic] += 1
        topics_words[topic,word2id[word]] += 1
    topic_assignments.append(doc_topics)
    

### Gibbs Sampling

In [7]:
def Gibbs_sampling(d,word_id,words_topics,docs_topics,topics_words,alpha,ita):
    
    topic_probs = (docs_topics[d] + alpha) / np.sum(docs_topics[d] + alpha)
    word_sum = np.sum(topics_words + ita, axis = 1)
    word_probs = (words_topics[word_id] + ita[word_id]) / word_sum
    # posterior probs
    probs = topic_probs * word_probs
    # normalize
    sample_probs = probs / np.sum(probs)
    #print(sample_probs)
    # sample new topic for current word
    new_topic = list(np.random.multinomial(1, sample_probs, size=1)[0]).index(1)
    return new_topic

In [8]:
import copy
stop = 1
i = 0
num_iterations = 9
for j in range(num_iterations):
    for d in range(len(corpus)):
        document = corpus[d]
        for n in range(len(document)):
            word = document[n]
            word_id = word2id[word]
            topic = topic_assignments[d][n]
            # exclude current word and topic
            docs_topics[d][topic] -= 1
            topics_words[topic][word_id] -=1
            words_topics[word_id,topic] -= 1
            new_topic = Gibbs_sampling(d,word_id,words_topics,docs_topics,topics_words,alpha,ita)
            # update topic and word state
            docs_topics[d][new_topic] += 1
            topics_words[new_topic][word_id] += 1
            words_topics[word_id,new_topic] += 1
            topic_assignments[d][n] = new_topic

### Evaluation

In [9]:
docs_topics

array([[ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.,  0.],
       [ 0.,  0., 10.],
       [ 0., 10.,  0.],
       [10.,  0.

In [15]:
import matplotlib.pyplot as plt
for i,state in enumerate(topics_words):
    # sorted descending word frequence within each topic
    topic_id_freq = sorted(range(len(state)), key=lambda k: state[k], reverse=True)
    topic_word_freq = [id2word[i] for i in topic_id_freq]
    print("Topic: ", i)
    print(topic_word_freq)

Topic:  0
['speed', 'strong', 'chelase', 'stadium', 'dream', 'ronald', 'player', 'messi', 'attack', 'hard', 'fast', 'liverpool', 'ozil', 'manU', 'win', 'pitch', 'play', 'defense', 'cup', 'shot', 'practice', 'ballon', 'move', 'football', 'lunch', 'vegetable', 'lion', 'fish', 'peach', 'banana', 'micky', 'munching', 'tea', 'mice', 'eat', 'pig', 'piggy', 'pork', 'breakfast', 'chicken', 'rabbit', 'beef', 'bird', 'ham', 'beacon', 'spinach', 'apple', 'tiger', 'rice', 'cute', 'horse', 'cream', 'pan', 'chinchilla', 'smoothie', 'dragon', 'KFC', 'dinner', 'zoo', 'cat', 'zoology', 'dog', 'restaurant', 'broccoli', 'monkey', 'panda', 'noodle', 'bee', 'duck', 'hamster', 'elephant', 'kitten']
Topic:  1
['lion', 'chinchilla', 'horse', 'duck', 'mice', 'bird', 'cat', 'micky', 'rabbit', 'dragon', 'panda', 'bee', 'piggy', 'tiger', 'zoo', 'elephant', 'cute', 'hamster', 'munching', 'dog', 'pig', 'zoology', 'fish', 'kitten', 'monkey', 'vegetable', 'ozil', 'peach', 'messi', 'banana', 'win', 'lunch', 'practice'

In [11]:
topics_words

array([[ 0.,  0.,  0., 42.,  0., 44.,  0.,  0.,  0., 41.,  1., 32., 44.,
        37.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 36.,  0.,  0., 46.,  0.,
         0.,  0., 35.,  0., 48.,  0.,  0.,  0.,  0.,  0., 32.,  0.,  0.,
        44.,  0.,  0., 42.,  0.,  0.,  0.,  0., 40.,  0.,  0., 47., 29.,
         0., 51.,  0., 43., 32.,  0., 44., 40., 52.,  0., 51.,  0., 47.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 50., 31.,  0.,  0.,  0.,  0., 42., 37.,  0.,  0.,  0.,  0.,
         0.,  0., 46.,  0., 33., 40.,  0.,  0.,  0.,  0., 42.,  0.,  0.,
        43.,  0.,  0.,  0.,  0.,  0.,  0., 40.,  0., 38.,  0., 48.,  0.,
         0.,  0., 49.,  0.,  0., 42.,  0.,  0.,  0., 39., 43.,  0.,  0.,
        33.,  0., 37.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 29.,  0.,
        42.,  0., 41., 47., 38., 39., 31.],
       [43.,  0.,  0.,  0., 45.,  0., 38.,  0.,  0.,  0., 35.,  0.,  0.,
         0., 49.,  0., 53.,  0.,  0., 39., 41.,  0., 44.,  0.,  0., 36.,
         0., 41.,  1., 34.,  0., 40.

In [17]:
for i in range(len(words_topics)):
    print(words_topics[i],id2word[i])

[ 0.  0. 43.] vegetable
[ 0. 50.  0.] lion
[ 0. 31.  0.] fish
[42.  0.  0.] ozil
[ 0.  0. 45.] peach
[44.  0.  0.] messi
[ 0.  0. 38.] banana
[ 0. 42.  0.] micky
[ 0. 37.  0.] munching
[41.  0.  0.] win
[ 1.  0. 35.] lunch
[32.  0.  0.] practice
[44.  0.  0.] attack
[37.  0.  0.] defense
[ 0.  0. 49.] tea
[ 0. 46.  0.] mice
[ 0.  0. 53.] eat
[ 0. 33.  0.] pig
[ 0. 40.  0.] piggy
[ 0.  0. 39.] pork
[ 0.  0. 41.] breakfast
[36.  0.  0.] cup
[ 0.  0. 44.] chicken
[ 0. 42.  0.] rabbit
[46.  0.  0.] player
[ 0.  0. 36.] beef
[ 0. 43.  0.] bird
[ 0.  0. 41.] ham
[35.  0.  1.] shot
[ 0.  0. 34.] beacon
[48.  0.  0.] stadium
[ 0.  0. 40.] spinach
[ 0.  0. 39.] apple
[ 0. 40.  0.] tiger
[ 0.  0. 52.] rice
[ 0. 38.  0.] cute
[32.  0.  0.] ballon
[ 0. 48.  0.] horse
[ 0.  0. 81.] cream
[44.  0.  0.] hard
[ 0.  0. 40.] pan
[ 0. 49.  0.] chinchilla
[42.  0.  0.] manU
[ 0.  0. 46.] smoothie
[ 0. 42.  0.] dragon
[ 0.  0. 41.] KFC
[ 0.  0. 37.] dinner
[40.  0.  0.] pitch
[ 0. 39.  0.] zoo
[ 0. 43.  0.

### Comparison
Justify my result with gensim LDA model

In [18]:
import gensim
from gensim import corpora
text_data = corpus
dictionary = corpora.Dictionary(text_data)
id_corpus = [dictionary.doc2bow(text) for text in text_data]

ldamodel = gensim.models.ldamodel.LdaModel(id_corpus, num_topics = num_topics, id2word=dictionary, passes=12)
#ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=num_words)
for topic in topics:
    print(topic)

(0, '0.049*"lion" + 0.048*"chinchilla" + 0.047*"horse" + 0.046*"duck" + 0.045*"mice" + 0.042*"bird" + 0.042*"cat" + 0.041*"panda" + 0.041*"dragon" + 0.041*"micky" + 0.041*"rabbit" + 0.040*"bee" + 0.039*"tiger" + 0.039*"piggy" + 0.038*"elephant" + 0.038*"zoo" + 0.037*"cute" + 0.037*"hamster" + 0.036*"dog" + 0.036*"munching" + 0.033*"zoology" + 0.033*"pig" + 0.031*"fish" + 0.031*"kitten" + 0.029*"monkey" + 0.000*"breakfast" + 0.000*"strong" + 0.000*"rice" + 0.000*"chelase" + 0.000*"tea" + 0.000*"ham" + 0.000*"cream" + 0.000*"chicken" + 0.000*"pork" + 0.000*"restaurant" + 0.000*"dinner" + 0.000*"vegetable" + 0.000*"spinach" + 0.000*"broccoli" + 0.000*"smoothie" + 0.000*"fast" + 0.000*"banana" + 0.000*"hard" + 0.000*"speed" + 0.000*"win" + 0.000*"dream" + 0.000*"practice" + 0.000*"pan" + 0.000*"eat" + 0.000*"manU" + 0.000*"beacon" + 0.000*"pitch" + 0.000*"liverpool" + 0.000*"move" + 0.000*"play" + 0.000*"ozil" + 0.000*"noodle" + 0.000*"player" + 0.000*"peach" + 0.000*"lunch" + 0.000*"KFC" 